# Get-RscMssqlDatabaseFiles
When DBAs do a restore of a database, they often have to change the location of where the restored database will sit. They do this by first running RESTORE FILELISTONLY to see where the files of the database existed at the time of the backup. They use this information when building their RESTORE DATABASE statements. 

# Workflow
To get information about MSSQL Database files, you need to do the following
1. Use **Connect-RSC** to Connect to RSC
1. Use **Get-RscCluster** to get the cluster id that the SQL Server is registered. This is akin to the datacenter that the SQL Server is located in. While you can run the next command without the cluster ID, you may get more records back than expected. This is because your SQL Server Host could exist on 2 different cluster. Once as a replicated source and once as the replicated target. 
1. Use **Get-RscMssqlInstance** to get the instance id of the SQL Server based on the name of the instance and the cluster where it is located. 
1. Use **Get-RscMssqlDatabase** to get the database ID based on the database name and the SQL Server Instance. 
1. Use **Get-RscMssqlDatabaseRecoveryPoint** to get the fully formatted date that corresponds to the recovery point selected.
1. Use **Get-RscMssqlDatabaseFiles** to get the list of files from the backup. 
1. Use **Disconnect-RSC** to disconnect from RSC

## Connect to RSC

In [1]:
Import-Module RubrikSecurityCloud
Connect-RSC


ClientId                                AuthenticationState SdkSchemaVersion ServerVersion
--------                                ------------------- ---------------- -------------
client|kB5og68XXrFlfXcvWhn3jlHdf1OKyJCB          AUTHORIZED v20231211-26     v20231211-26



## Get the Cluster ID

In [2]:
$ClusterName = "vault-r-london"
$RscCluster = Get-RscCluster -Name $ClusterName

## Get the Instance Information

In [3]:
$HostName = "rp-sql19s-001.perf.rubrik.com"
$RscHost = Get-RscHost -Name $HostName -OsType Windows
$RscMssqlInstance = Get-RscMssqlInstance -RscHost $RscHost -RscCluster $RscCluster

## Get the Database Information

In [5]:
$DatabaseName = "AdventureWorks2019"
$RscMssqlDatabase = Get-RscMssqlDatabase -Name $DatabaseName  -RscMssqlInstance $RscMssqlInstance

## Get the fully formatted recovery point

In [6]:
$RecoveryDateTime = Get-RscMssqlDatabaseRecoveryPoint -RscMssqlDatabase $RscMssqlDatabase -Latest

## Get list of files of the database from the backup

In [7]:
$RscMssqlDatabaseFiles = Get-RscMssqlDatabaseFiles -RscMssqlDatabase $RscMssqlDatabase -RecoveryDateTime $RecoveryDateTime
$RscMssqlDatabaseFiles


FileType     : MSSQL_DATABASE_FILE_TYPE_DATA
FileId       : 1
LogicalName  : AdventureWorks2017
OriginalName : AdventureWorks2019.mdf
OriginalPath : c:\mnt\sqldata

FileType     : MSSQL_DATABASE_FILE_TYPE_LOG
FileId       : 2
LogicalName  : AdventureWorks2017_log
OriginalName : AdventureWorks2019_log.ldf
OriginalPath : c:\mnt\sqllogs



---

# Disconnect from RSC
It is important to have each script you create end by disconnecting from RSC. If you do not, then you will run into issues with token authentication exhaustion. RSC allows a Service Account to have 10 concurrent open connections to RSC. Additional requests will be denied until existing connections are closed. 

You can easily disconnect your connection by running the below statement. 

In [ ]:
Disconnect-Rsc